In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO

In [ ]:
VIDEO_INPUT = "test.mp4"
VIDEO_OUTPUT = "video_result.mp4"
CSV_OUTPUT = "resultados.csv"
YOLO_GENERAL = "yolo11n.pt"           # Modelo general: personas y vehículos
YOLO_LP = "runs/detect/train4/weights/best.pt"  # Modelo para matrículas
CONF_THRESHOLD = 0.3

# Inicializar modelos
model_general = YOLO(YOLO_GENERAL)
model_lp = YOLO(YOLO_LP)

# Preparar vídeo de salida
cap = cv2.VideoCapture(VIDEO_INPUT)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(VIDEO_OUTPUT, fourcc, fps, (width, height))

# Lista para resultados
data = []
frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # Detección + tracking de personas y vehículos
    results = model_general.track(frame, persist=True, conf=CONF_THRESHOLD)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            obj_id = box.id
            cls_name = model_general.names[int(box.cls)]  # persona o vehículo
            conf = float(box.conf)

            # Dibujar persona o vehículo con su propia etiqueta
            color = (0, 255, 0) if cls_name == "person" else (0, 0, 255)
            label = f"{cls_name}-{obj_id}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Detectar matrículas en el frame completo
            if cls_name in ["car", "truck"]:
                lp_results = model_lp.track(frame, persist=True, conf=0.1)

                if lp_results and len(lp_results) > 0:
                    for lp_r in lp_results:
                        for lp_box in lp_r.boxes:
                            lx1, ly1, lx2, ly2 = map(int, lp_box.xyxy[0].tolist())

                            # Filtrar matrículas dentro del vehículo
                            if lx1 >= x1 and ly1 >= y1 and lx2 <= x2 and ly2 <= y2:
                                lp_coords = (lx1, ly1, lx2, ly2)
                                lp_id = lp_box.id  # ID asignado automáticamente por YOLO
                                lp_conf = float(lp_box.conf)

                                # Guardar resultados en CSV
                                data.append({
                                    "fotograma": frame_idx,
                                    "tipo_objeto": "matricula",
                                    "confianza": lp_conf,
                                    "identificador_tracking": lp_id,
                                    "x1": lp_coords[0],
                                    "y1": lp_coords[1],
                                    "x2": lp_coords[2],
                                    "y2": lp_coords[3]
                                })

                                # Dibujar matrícula con su propio ID
                                cv2.rectangle(frame, (lp_coords[0], lp_coords[1]),
                                              (lp_coords[2], lp_coords[3]), (255, 255, 0), 2)
                                cv2.putText(frame, f"matricula-{lp_id}", 
                                            (lp_coords[0], lp_coords[1]-10),
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    # Guardar frame procesado
    out.write(frame)
    cv2.imshow("Resultados", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Guardar CSV
df = pd.DataFrame(data)
df.to_csv(CSV_OUTPUT, index=False)

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print("========= Procesamiento completado =========")
print(f"CSV guardado en: {CSV_OUTPUT}")
print(f"Vídeo resultante guardado en: {VIDEO_OUTPUT}")
